In [1]:
from load_dataset import load_data
import numpy as np
import torch
from torch.utils.data import TensorDataset,DataLoader
from UNet import UNet
from conbine_loss import CombinedLoss
import torch.optim as optim
import torch.nn as nn
path_test = "../input0/test_npz_seg_512x512_s1"
path_train ='../input0/train_npz_seg_512x512_s1'
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [2]:
torch.cuda.is_available()

True

In [2]:
np_test,np_test_mask = load_data(path_test)
np_train,np_train_mask = load_data(path_train)
# print(np_test.shape,np_test_mask.shape)
# print(np_train.shape,np_train_mask.shape)
# 归一化
test_data = (np_test/255).astype(np.float32)
test_mask = np_test_mask
train_data = (np_train/255).astype(np.float32)
train_mask = np_train_mask
train_data =torch.from_numpy(train_data)
train_data = train_data.permute(0, 3, 1, 2)
test_data = torch.from_numpy(test_data)
test_data = test_data.permute(0, 3, 1, 2)
## print(train_data.shape)
## print(train_mask.shape)


In [3]:
test_set = TensorDataset(test_data, torch.from_numpy(test_mask))
train_set = TensorDataset(train_data, torch.from_numpy(train_mask))

net = UNet(n_channels=3,n_classes=26).to(device)
criterion = CombinedLoss()#定义损失函数
optimizer = optim.Adam(net.parameters(), lr=0.010)  # 定义优化器
trainLoader = DataLoader(train_set, batch_size=2,
                         shuffle=True, num_workers=0)
testLoader = DataLoader(test_set, batch_size=1,
                         shuffle=False, num_workers=0)


In [ ]:
import time
start = time.time()
for epoch in range(150):
    running_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        inputs, labels = data  # 获取数据
        inputs = inputs.to(device)
        labels = labels.to(device)
        # print(inputs.shape)
        optimizer.zero_grad()  # 清空梯度缓存
        outputs = net(inputs)
        # print(outputs.shape)
        loss = criterion(outputs, labels.long())
        loss.backward()  # 反向传播
        optimizer.step()  # 调整模型
        running_loss += loss.item()
        if i % 10 == 9:
            # 每 20 次迭代打印一次信息
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss))
            running_loss = 0.0
print('Finish Traning! Total cost time: ', time.time()-start)



[1,    10] loss: 12.084
[1,    20] loss: 6.995
[1,    30] loss: 6.966
[1,    40] loss: 6.622
[1,    50] loss: 6.562
[1,    60] loss: 6.720
[1,    70] loss: 6.503
[1,    80] loss: 6.649
[1,    90] loss: 6.386
[1,   100] loss: 6.460
[1,   110] loss: 6.662
[1,   120] loss: 6.672
[1,   130] loss: 6.604
[1,   140] loss: 6.429
[1,   150] loss: 6.442
[1,   160] loss: 6.548
[1,   170] loss: 6.467
[1,   180] loss: 6.566
[1,   190] loss: 6.465
[1,   200] loss: 6.442
[1,   210] loss: 6.389
[1,   220] loss: 6.402
[1,   230] loss: 6.455
[1,   240] loss: 6.342
[1,   250] loss: 6.492
[1,   260] loss: 6.436
[1,   270] loss: 6.473
[2,    10] loss: 6.458
[2,    20] loss: 6.519
[2,    30] loss: 6.338
[2,    40] loss: 6.380
[2,    50] loss: 6.418
[2,    60] loss: 6.325
[2,    70] loss: 6.397
[2,    80] loss: 6.459
[2,    90] loss: 6.326
[2,   100] loss: 6.349
[2,   110] loss: 6.369
[2,   120] loss: 6.332
[2,   130] loss: 6.528
[2,   140] loss: 6.405
[2,   150] loss: 6.489
[2,   160] loss: 6.340
[2,   170]

In [ ]:
torch.save(net, 'dice_net.pkl ')